In [1]:
import os
import json
import pandas as pd
from modules import spotify_methods as sm
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

1.1) First we need a list of songs to compare to our training data

User ID and Playlist ID can be found on spotify via a web browser

In [2]:
new_playlist = sm.collect_tracks_with_metadata(["6xUpKQJ6VdZ6j3UEbwV3Zl"], "tmmb1pzyny780errk5d7vuz6r", 100000)
# 6xUpKQJ6VdZ6j3UEbwV3Zl
columns = ['pid']
columns += [np[1] for np in new_playlist]

 #avoid dupelicates
test_df = pd.DataFrame(columns = columns)

In [3]:
test_df

,pid,my songs know what you did in the dark (light em up),the last of the real ones,hold me tight or don’t,wilson (expensive mistakes),church,heaven’s gate,champion,sunshine riptide,young and menace,...,nobody,watch me burn,heart attack - rock version,fake out,hold me like a grudge,heartbreak feels so good,anx13ty,cyan1de (feat. pete wentz),horror show,holiday


1.2) Collect training data

We limit the dataset collection to allow for a higher chance of picking up relationships within data we are familiar with

In [4]:
dataset_path = "C:\\Users\\Shivesh\\Documents\\Repos\\Personal\\Projects\\music-recommendor\\millionplaylistsubset"

files_count = 0
filenames = os.listdir(dataset_path)

collect_songs_dict = {}
max_id = 0
for filename in sorted(filenames):

    if filename.startswith("mpd.slice.") and filename.endswith(".json"):

        fullpath = os.sep.join((dataset_path, filename))
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)

        for playlist in mpd_slice["playlists"]:

            if(playlist.get("pid") > 10):
                break
            
            collect_songs_dict[playlist.get("pid")] = {}
            tracks = playlist['tracks']

            for track in tracks:

                collect_songs_dict[playlist.get("pid")][track.get("track_name").lower()]  = track.get("artist_name").lower()

            
            max_id +=1

        files_count += 1

print(files_count)
print(max_id)
# print(collect_songs_dict)

100
11


In [5]:
# spotify_users = ['tmmb1pzyny780errk5d7vuz6r', '1yoo33uf0br9bvkmn1wlahjgr', 'mgscx8hs8egcjamt31rfkm8ww']
spotify_users = ['1yoo33uf0br9bvkmn1wlahjgr', 'mgscx8hs8egcjamt31rfkm8ww']
extra_playlists = {}
for su in spotify_users:

    user_playlists = sm.collect_user_playlists(su)
    extra_playlists[su] = user_playlists


for ep_user, ep_playlists in extra_playlists.items():

    collect_tracks = sm.collect_dict_tracks(ep_playlists, ep_user, max_id)
    max_id += len(ep_playlists)
    collect_songs_dict.update(collect_tracks)

In [6]:
print(collect_songs_dict.get(1))

{'eye of the tiger': 'survivor', 'libera me from hell (tengen toppa gurren lagann)': 'daniel tidwell', 'pokémon theme': 'daniel tidwell', 'concerning hobbits (the lord of the rings)': 'daniel tidwell', 'the blood of cuchulainn (the boondock saints)': 'daniel tidwell', "he's a pirate (pirates of the caribbean)": 'daniel tidwell', "very bloody tears (castlevania ii: simon's quest)": 'daniel tidwell', 'u.n. owen was her? (remix)': 'kaleptik', 'i am the doctor in utah': 'ben foster', 'the room where it happens': 'leslie odom jr.', 'right hand man': 'christopher jackson', 'alexander hamilton': 'leslie odom jr.', 'my shot': 'lin-manuel miranda', 'stairway to heaven': 'led zeppelin', 'shine': 'collective soul', 'ghost love score': 'nightwish', 'crazy': 'seal', 'sympathy for the devil': 'the rolling stones', 'gimme shelter': 'the rolling stones', 'free bird': 'lynyrd skynyrd', 'peace of mind': 'boston', 'foreplay / long time': 'boston', 'hold the line': 'toto', 'carry on wayward son': 'kansas'

1.3) See which users from the training data has listened to the songs we're interested in

In [7]:
song_columns = columns[1:]

for id in collect_songs_dict.keys():
    
    row = {}
    row['pid'] = id
    for s in song_columns:

        value = 0
        if(collect_songs_dict.get(id).get(s)):
            value = 1
        
        row[s] = value

    test_df = test_df.append(row, ignore_index = True)

In [8]:
print(test_df.iloc[0])

pid                                                     0
my songs know what you did in the dark (light em up)    0
the last of the real ones                               0
hold me tight or don’t                                  0
wilson (expensive mistakes)                             0
                                                       ..
heartbreak feels so good                                0
anx13ty                                                 0
cyan1de (feat. pete wentz)                              0
horror show                                             0
holiday                                                 0
Name: 0, Length: 280, dtype: object


In [9]:
del test_df['pid']

In [37]:
frequent_itemsets = apriori(test_df, min_support=0.11, use_colnames=True)

In [38]:
frequent_itemsets

,support,itemsets
0,0.117647,(misery business)
1,0.117647,(all the small things)
2,0.117647,(mr. brightside)
3,0.117647,(wolf in sheep's clothing)
4,0.117647,(crazy)
...,...,...
630,0.117647,"(misery business, forget me too (feat. halsey)..."
631,0.117647,"(bloody valentine, misery business, forget me ..."
632,0.117647,"(misery business, bloody valentine, my ex's be..."
633,0.117647,"(bloody valentine, misery business, forget me ..."


In [59]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=8.5)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
pd.set_option('display.max_rows', 200)
rules[ (rules['antecedent_len'] >= 5) &
       (rules['confidence'] > 0.75)
       ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
3080,"(misery business, bloody valentine, forget me ...",(my ex's best friend (with blackbear)),0.117647,0.117647,0.117647,1.0,8.5,0.103806,inf,5
3081,"(misery business, bloody valentine, my ex's be...",(forget me too (feat. halsey)),0.117647,0.117647,0.117647,1.0,8.5,0.103806,inf,5
3082,"(misery business, forget me too (feat. halsey)...",(bloody valentine),0.117647,0.117647,0.117647,1.0,8.5,0.103806,inf,5
3083,"(bloody valentine, forget me too (feat. halsey...",(misery business),0.117647,0.117647,0.117647,1.0,8.5,0.103806,inf,5
3136,"(misery business, bloody valentine, forget me ...",(my ex's best friend (with blackbear)),0.117647,0.117647,0.117647,1.0,8.5,0.103806,inf,5
...,...,...,...,...,...,...,...,...,...,...
6052,"(forget me too (feat. halsey), my ex's best fr...","(bloody valentine, misery business, title track)",0.117647,0.117647,0.117647,1.0,8.5,0.103806,inf,5
6053,"(forget me too (feat. halsey), my ex's best fr...","(bloody valentine, misery business, all i know...",0.117647,0.117647,0.117647,1.0,8.5,0.103806,inf,5
6054,"(forget me too (feat. halsey), my ex's best fr...","(bloody valentine, misery business, drunk face)",0.117647,0.117647,0.117647,1.0,8.5,0.103806,inf,5
6055,"(forget me too (feat. halsey), drunk face, all...","(bloody valentine, my ex's best friend (with b...",0.117647,0.117647,0.117647,1.0,8.5,0.103806,inf,5
